In [2]:
import datetime
import gpm
from gpm.utils.geospatial import get_country_extent
import os
import h5py
import shutil

In [3]:
# output_dir = r'C:\Users\c.kwa\Desktop\meteosat_retrieval\IMERG_retrieval\Resampled_nat\2020'
output_dir_cropped_imerg_f = r'D:\Ghana\IMERG_data\test_data\cropped\IMERG-FR'
output_dir_cropped_imerg_e = r'D:\Ghana\IMERG_data\test_data\cropped\IMERG-ER'
input_dir_imerg_f = r'D:\Ghana\IMERG_data\GPM\RS\V07\IMERG\IMERG-FR'
input_dir_imerg_e = r'D:\Ghana\IMERG_data\GPM\NRT\IMERG\IMERG-ER'
output_dir_copy_imerg_f = r'C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\RS\V07\IMERG\IMERG-FR'
output_dir_copy_imerg_e = r'C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER'

test_dir = r"Z:\cluster_projects\ro\1149_10\earthformer-multisource-to-inca\data_adapted_condition_0.02_20_all\test"

In [4]:
def extract_timestamp_test(filename):
    time = filename.split('.')[0].split('_')[1]
    dt = datetime.datetime.strptime(time, "%Y%m%d%H%M%S")
    return dt

def extract_timestamp_imerg(filename):
    day = filename.split('.')[4].split('-')[0]
    hoday = filename.split('.')[4].split('-')[2][1:7]
    dt = datetime.datetime.strptime(day + hoday, "%Y%m%d%H%M%S")
    return dt

In [5]:
date_list = []

for file in os.listdir(test_dir):
    date_list.append(extract_timestamp_test(file))   

In [6]:
def copy_files(date_list, input_dir, output_dir):
    # Traverse through all subdirectories and files in the given path
    #file_path = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".HDF5") and (extract_timestamp_imerg(file) in date_list):
                day = file.split('.')[4].split('-')[0][6:8]
                month = file.split('.')[4].split('-')[0][4:6]
                year = file.split('.')[4].split('-')[0][0:4]                
                srcs = os.path.join(root, file)
                file_dir = os.path.join(output_dir, year, month, day)

                os.makedirs(file_dir, exist_ok = True)

                filepath = fr'{file_dir}\\{file}'

                print(filepath)
                # Copy File
                shutil.copy(srcs, filepath)
                print(f'{file} copied!')
copy_files(date_list, input_dir_imerg_e, output_dir_copy_imerg_e)

C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER\2020\03\03\\3B-HHR-E.MS.MRG.3IMERG.20200303-S100000-E102959.0600.V06B.HDF5
3B-HHR-E.MS.MRG.3IMERG.20200303-S100000-E102959.0600.V06B.HDF5 copied!
C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER\2020\03\03\\3B-HHR-E.MS.MRG.3IMERG.20200303-S180000-E182959.1080.V06B.HDF5
3B-HHR-E.MS.MRG.3IMERG.20200303-S180000-E182959.1080.V06B.HDF5 copied!
C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER\2020\03\03\\3B-HHR-E.MS.MRG.3IMERG.20200303-S210000-E212959.1260.V06B.HDF5
3B-HHR-E.MS.MRG.3IMERG.20200303-S210000-E212959.1260.V06B.HDF5 copied!
C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER\2020\03\06\\3B-HHR-E.MS.MRG.3IMERG.20200306-S180000-E182959.1080.V06B.HDF5
3B-HHR-E.MS.MRG.3IMERG.20200306-S180000-E182959.1080.V06B.HDF5 copied!
C:\Users\c.kwa\Desktop\meteosat_retrieval\Benchmarking\GPM\NRT\IMERG\IMERG-ER\2020\03\07\\3B-HHR-E.MS.MR

KeyboardInterrupt: 

In [7]:
# Load the SEVIRI file
import xarray as xr
file_name = 'MSG4-SEVI-MSG15-0100-NA-20200501001242.772000000Z-NA.hdf5'
file_path = fr'C:\Users\c.kwa\Desktop\meteosat_retrieval\SEVIRI_retrieval\Test_batch\Native_to_h5\hdf5\2020\05\{file_name}'
seviri_ds = xr.open_dataset(file_path, engine = 'netcdf4')

# Bounding box (in degrees)
min_lon = seviri_ds['x'].min().item()
max_lon = seviri_ds['x'].max().item()
min_lat = seviri_ds['y'].min().item()
max_lat = seviri_ds['y'].max().item()
print(min_lon, max_lon, min_lat, max_lat)

-3.6862771739130435 1.336277173913044 4.513709677419355 11.286290322580646


In [8]:
def get_data(start_time, end_time, product="IMERG-FR", product_type="RS", version=7):# Specify the time period you are interested in

    # - If chunks is not None, it does not load the data in RAM memory !
    ds = gpm.open_dataset(
        product=product,
        product_type=product_type,
        version=version,
        start_time=start_time,
        end_time=end_time,
        #variables=["precipitationCal", "probabilityLiquidPrecipitation"],
    )
    return ds

In [9]:
def save_cropped_file(ds, output_dir, dataset_name):
    # Loop through all time steps in the dataset
    for i in range(ds.dims['time']):
        # Extract data for the current time step
        ds_at_time = ds.isel(time=i)
        
        # Extract the time as a datetime object
        time_value = pd.to_datetime(ds_at_time.time.values)
        
        # Construct the filename based on the desired format
        end_time_str = (time_value - pd.Timedelta(seconds=1)).strftime("E%H%M%S")
        start_time_str = (time_value - pd.Timedelta(minutes=30)).strftime("S%H%M%S")  # Assuming 30 min intervals
        date_str = pd.to_datetime(time_value - pd.Timedelta(minutes=30)).strftime("%Y%m%d")

        filename = f"3B-HHR.MS.MRG.3IMERG.{date_str}-{start_time_str}-{end_time_str}.0000.V07B.cropped.HDF5"
        
        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        filepath = os.path.join(output_dir, filename)

        if not os.path.exists(filepath):
            # Save the data as an HDF5 file
            with h5py.File(filepath, 'w') as new_file:
                for var_name in ds_at_time.data_vars:
                    # Save each variable as a dataset
                    new_file.create_dataset(var_name, data=ds_at_time[var_name].values)

                # # Save latitude and longitude grids if they exist
                # if 'lon' in ds_at_time.coords and 'lat' in ds_at_time.coords:
                #     new_file.create_dataset('lon', data=ds_at_time['lon'].values)
                #     new_file.create_dataset('lat', data=ds_at_time['lat'].values)

                # Save time information as an attribute
                new_file.attrs['time'] = str(time_value)

                print(f"Saved cropped file: {filename}")

In [11]:
for time in date_list:
    start_time = time - datetime.timedelta(minutes=30)
    start_time = datetime.datetime.strptime(str(start_time), "%Y-%m-%d %H:%M:%S")
    print(start_time)
    end_time = time + datetime.timedelta(seconds = 30)
    end_time = datetime.datetime.strptime(str(end_time), "%Y-%m-%d %H:%M:%S")
    print(end_time)
    ds = get_data(start_time, end_time, "IMERG-FR", "RS", 7)
    ds_ghana = ds.gpm.crop(extent=[min_lon, max_lon, min_lat, max_lat])
    save_cropped_file(ds_ghana, output_dir_cropped_imerg_f, "IMERG-FR")

TypeError: list indices must be integers or slices, not tuple

In [15]:
start_time = date_list[0] - datetime.timedelta(minutes=30)
start_time = datetime.datetime.strptime(str(start_time), "%Y-%m-%d %H:%M:%S")
print(start_time)
end_time = date_list[0] + datetime.timedelta(seconds = 30)
end_time = datetime.datetime.strptime(str(end_time), "%Y-%m-%d %H:%M:%S")
print(end_time)
ds = get_data(start_time, end_time, "IMERG-FR", "RS", 7)
ds_ghana = ds.gpm.crop(extent=[min_lon, max_lon, min_lat, max_lat])

2023-11-24 21:29:59
2023-11-24 22:00:29


C:\Users\c.kwa\AppData\Local\miniforge3\envs\gpm-api-py311\Lib\site-packages\gpm\dataset\conventions.py:82: GPM_Warning: 'The dataset end_time 2023-11-24 22:00:00 occurs before the specified end_time 2023-11-24 22:00:29. Some granules may be missing!'
  warnings.warn(msg, GPM_Warning, stacklevel=1)


In [25]:
with h5py.File('grid.h5', 'w') as new_file:
    new_file.create_dataset('lon', data = ds_ghana['lon'])
    new_file.create_dataset('lat', data = ds_ghana['lat'])

In [30]:
# Specify the time period you are interested in
start_time = datetime.datetime.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

ds = get_data(start_time, end_time, "IMERG-FR", "RS", 7)

ValueError: No files found on disk. Please download them before.